# Lab 4: Logic App Integration

In this lab, you will learn how to integrate Azure Logic Apps as tools for your AI agent, enabling complex workflow orchestration.

## Learning Objectives

- Understand Logic Apps as integration tools
- Create a Logic App tool definition
- Handle workflow-based tool responses
- Combine Functions and Logic Apps in one agent

## Prerequisites

- Completed Labs 1-3
- Azure Logic App created (see [Logic Apps Setup Guide](../docs/logic_apps_setup.md))
- `.env` file configured with `LOGIC_APP_TRIGGER_URL`

## Step 1: Import Required Modules

In [ ]:
from northwestern_foundry_agent import Settings, FoundryAgent
from northwestern_foundry_agent.foundry.tools import (
    create_health_check_tool,
    create_quote_tool,
    create_logic_app_tool,
)
from northwestern_foundry_agent.integrations.azure_functions import (
    AzureFunctionsClient,
    create_function_handlers,
)
from northwestern_foundry_agent.integrations.logic_apps import (
    LogicAppsClient,
    LogicAppRequest,
    create_logic_app_handler,
)
from northwestern_foundry_agent.utils import setup_logging, get_logger

setup_logging(Settings(log_level="INFO", log_format="text"))
logger = get_logger(__name__)

print("Modules imported successfully!")

## Step 2: Load Configuration

In [ ]:
settings = Settings()

print(f"Azure AI configured: {settings.is_configured}")
print(f"Functions configured: {settings.functions_configured}")
print(f"Logic Apps configured: {settings.logic_apps_configured}")

if not settings.logic_apps_configured:
    print("\n⚠️ Warning: Logic Apps not configured.")
    print("Update .env with LOGIC_APP_TRIGGER_URL")

## Step 3: Understand Logic App Request/Response

Let's examine the data models for Logic App communication.

In [ ]:
import json

# Create a sample request
sample_request = LogicAppRequest(
    action="process_document",
    input_data={
        "document_id": "doc-12345",
        "document_type": "invoice",
    },
    correlation_id="corr-abc-123",
    metadata={"source": "ai-agent", "priority": "normal"},
)

print("Sample Logic App Request:")
print(json.dumps(sample_request.to_trigger_payload(), indent=2, default=str))

print("\n" + "="*50)
print("\nExpected Response Schema:")
print(json.dumps({
    "workflow_run_id": "<unique-run-id>",
    "status": "succeeded | failed | running",
    "output_data": {"result": "<workflow output>"},
    "error": "<error message if failed>",
    "started_at": "<ISO timestamp>",
    "completed_at": "<ISO timestamp>",
}, indent=2))

## Step 4: Test Logic App Directly (Optional)

If you have a Logic App configured, test it directly.

In [ ]:
if settings.logic_apps_configured:
    try:
        logic_client = LogicAppsClient(settings)
        
        # Create a test request
        test_request = LogicAppRequest(
            action="test",
            input_data={"message": "Hello from notebook!"},
            correlation_id="notebook-test-001",
        )
        
        response = await logic_client.trigger(test_request)
        
        print("Logic App Response:")
        print(f"  Run ID: {response.workflow_run_id}")
        print(f"  Status: {response.status.value}")
        print(f"  Successful: {response.is_successful}")
        print(f"  Output: {response.output_data}")
    except Exception as e:
        print(f"Error calling Logic App: {e}")
else:
    print("Logic Apps not configured. Example response:")
    print("  Run ID: 08585123-4567-89ab-cdef")
    print("  Status: succeeded")
    print("  Output: {'result': 'processed'}")

## Step 5: Create Logic App Tool Definition

In [ ]:
# Create Logic App tool
logic_trigger_url = settings.logic_app_trigger_url or "https://example-logic.azurewebsites.net/trigger"
logic_tool = create_logic_app_tool(logic_trigger_url, "process_request")

print("Logic App Tool Definition:")
print(f"  Name: {logic_tool.name}")
print(f"  Description: {logic_tool.description}")
print(f"  Trigger URL: {logic_tool.trigger_url[:50]}...")

print("\nParameters:")
for param in logic_tool.parameters:
    print(f"  - {param.name}: {param.param_type} (required: {param.required})")

print("\nFunction Definition (JSON):")
print(json.dumps(logic_tool.to_function_definition(), indent=2))

## Step 6: Create Full Agent with All Tools

Now let's create an agent with all three tools: health check, quotes, and Logic App.

In [ ]:
AGENT_INSTRUCTIONS = """
You are a versatile assistant with access to multiple tools.

Available tools:

1. **health_check**: Check system health status
   - Use when asked about system status, health, or monitoring

2. **quote_of_the_day**: Get inspirational quotes
   - Parameters: category (motivation, wisdom, humor)
   - Use when asked for quotes, inspiration, or motivation

3. **process_request**: Trigger a workflow via Logic App
   - Parameters: input_data (object with workflow data)
   - Use when asked to process documents, run workflows, or handle complex tasks

Guidelines:
- Select the most appropriate tool based on the user's request
- You can use multiple tools if the request requires it
- For workflow processing, structure the input_data appropriately
- Report results clearly and concisely
"""

all_configured = settings.is_configured and settings.functions_configured and settings.logic_apps_configured

if all_configured:
    try:
        agent = FoundryAgent(settings)
        
        # Create clients
        func_client = AzureFunctionsClient(settings)
        logic_client = LogicAppsClient(settings)
        
        # Get handlers
        func_handlers = create_function_handlers(func_client)
        logic_handler = create_logic_app_handler(logic_client)
        
        # Create tool definitions
        health_tool = create_health_check_tool(f"{settings.azure_function_app_url}/api/health")
        quote_tool = create_quote_tool(f"{settings.azure_function_app_url}/api/quote")
        
        # Register all tools
        agent.register_tool(health_tool, func_handlers["health_check"])
        agent.register_tool(quote_tool, func_handlers["quote_of_the_day"])
        agent.register_tool(logic_tool, logic_handler)
        
        # Create agent
        created_agent = agent.create_agent(
            name="full-integration-agent",
            instructions=AGENT_INSTRUCTIONS,
        )
        
        print(f"Agent created with all tools!")
        print(f"Registered tools: {list(agent.tools.keys())}")
    except Exception as e:
        print(f"Error: {e}")
        agent = None
else:
    print("Not all services configured. Example tool registration:")
    print("  - health_check: Azure Function")
    print("  - quote_of_the_day: Azure Function")
    print("  - process_request: Logic App")
    agent = None

## Step 7: Test Full Integration

In [ ]:
test_queries = [
    "Check the system health",
    "Give me a wise quote for the day",
    "Process a document with ID doc-456 and type 'report'",
    "I need system status and some motivation",
]

if agent and agent.agent:
    for query in test_queries:
        print(f"\n{'='*60}")
        print(f"User: {query}")
        print(f"{'='*60}")
        try:
            response = agent.run(query)
            print(f"\nAgent: {response}")
        except Exception as e:
            print(f"Error: {e}")
else:
    print("Agent not available. Example interactions:")
    print("\n" + "="*60)
    print("User: Process a document with ID doc-456 and type 'report'")
    print("="*60)
    print("\n[Agent calls process_request(input_data={...})]")
    print("\nAgent: I've submitted the document for processing.")
    print("  - Document ID: doc-456")
    print("  - Type: report")
    print("  - Status: succeeded")
    print("  - Workflow Run ID: 08585xxx")

## Step 8: Clean Up

In [ ]:
if agent and agent.agent:
    try:
        agent.delete_agent()
        print("Agent deleted successfully!")
    except Exception as e:
        print(f"Error: {e}")

## Summary

In this lab, you learned how to:

1. ✅ Understand Logic Apps as workflow tools
2. ✅ Create Logic App tool definitions
3. ✅ Combine Azure Functions and Logic Apps in one agent
4. ✅ Handle complex multi-tool interactions

## Key Concepts

- **Azure Functions**: Best for stateless, quick operations
- **Logic Apps**: Best for workflows, integrations, and orchestration
- **Multi-tool agents**: Can intelligently select and combine tools

## What's Next?

You've completed all the labs! Here are some ideas to continue learning:

- Add more Azure Functions with different capabilities
- Create complex Logic App workflows with multiple steps
- Explore Azure AI Foundry's advanced features
- Build a complete application using these components